## Transit Crowding

In [1]:
import requests

In [2]:
import geopandas as gpd

In [3]:
# download California census tracts

url = 'https://connectsf-transitcrowding.sfcta.org/assets/connectsf/trncrowding.zip'

with open('trncrowding.zip', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [4]:
# Load the census tracts as a GeoDataFrame

trncrowding = gpd.read_file('zip://trncrowding.zip')  # 'zip://' prefix clarifies format

In [5]:
trncrowding.head()

,A,B,MODE,NAME,TP,LOAD,geometry
0,4000,4008,22,40_CBW,AM,0.277,"LINESTRING (6065001.695 2130313.85188, 6066666..."
1,4018,4008,22,40_VE,EV,0.049,"LINESTRING (6067765.62349 2128408.12983, 60666..."
2,103456,4008,22,40_CBW,AM,0.129,"LINESTRING (6064367.73359 2133807.4221, 606666..."
3,4000,13080,22,40_CBW,AM,0.129,"LINESTRING (6065001.695 2130313.85188, 6063748..."
4,4002,4026,18,40_18W,AM,0.000,"LINESTRING (6068167.11572 2127707.98838, 60690..."


In [6]:
# I don't understand what does these number means...maybe result from the projection??
print(trncrowding.geometry[0]) 

LINESTRING (6065001.695 2130313.85188, 6066666.77451 2130407.24515)


## Transit Analysis Zone

In [7]:
# Get TAZ data from jobpop data

url = 'https://connectsf-populationandjobs.sfcta.org/assets/connectsf/jobpop.zip'

with open('jobpop.zip', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [8]:
jobpop = gpd.read_file('zip://jobpop.zip')

In [9]:
jobpop.head()

,TAZ,AREALAND,AREAWATR,Nhood,nhood_num,lg_nhood,lg_nhd_num,SQ_MILE,geometry
0,1,931073,0,Visitacion Valley,31,Bayshore,9,0.03340,"POLYGON ((6007378.500054573 2086055.37391938, ..."
1,2,1825218,0,Ocean View/Ingleside,36,Outer Mission,10,0.06547,"POLYGON ((5992137.499592379 2086335.12355164, ..."
2,3,1360855,0,Visitacion Valley,31,Bayshore,9,0.04881,"POLYGON ((6004983.499220342 2086750.499936027,..."
3,4,1596024,0,Outer Mission/Mission Terrace,34,Outer Mission,10,0.05725,"POLYGON ((5994473.501130554 2087145.000032395,..."
4,5,1064261,0,Visitacion Valley,31,Bayshore,9,0.03818,"POLYGON ((6006265.000917536 2086079.750239403,..."


In [10]:
taz=jobpop[['TAZ','geometry']].copy()

In [11]:
taz.head()

,TAZ,geometry
0,1,"POLYGON ((6007378.500054573 2086055.37391938, ..."
1,2,"POLYGON ((5992137.499592379 2086335.12355164, ..."
2,3,"POLYGON ((6004983.499220342 2086750.499936027,..."
3,4,"POLYGON ((5994473.501130554 2087145.000032395,..."
4,5,"POLYGON ((6006265.000917536 2086079.750239403,..."


## Linking Transit Crowding and TAZ

In [12]:
merged = gpd.sjoin(trncrowding, taz, how='inner', op='contains')

In [13]:
print(taz.crs)
print(trncrowding.crs)

{'init': 'epsg:2227'}
{'init': 'epsg:2227'}


In [14]:
taz_proj = taz.copy()
taz_proj.crs = trncrowding.crs

In [15]:
print(taz_proj.crs)

{'init': 'epsg:2227'}


In [27]:
merged = gpd.sjoin(taz, trncrowding, how='inner', op='intersects')

In [28]:
merged.head()

,TAZ,geometry,index_right,A,B,MODE,NAME,TP,LOAD
0,1,"POLYGON ((6007378.500054573 2086055.37391938, ...",13286,33605,20375,11,MUN9I,AM,0.110
5,6,"POLYGON ((6008496.000471613 2086032.624223359,...",13286,33605,20375,11,MUN9I,AM,0.110
0,1,"POLYGON ((6007378.500054573 2086055.37391938, ...",13294,20375,33605,11,MUN9O,AM,0.093
5,6,"POLYGON ((6008496.000471613 2086032.624223359,...",13294,20375,33605,11,MUN9O,AM,0.093
0,1,"POLYGON ((6007378.500054573 2086055.37391938, ...",13285,20369,20375,11,MUN9O,AM,0.093


In [29]:
len(merged)

18230

In [30]:
len(taz)

981

In [31]:
len(trncrowding)

25526

In [ ]:
#panda function to keep uniqle column index_right